In [10]:

# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정


In [11]:

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)

        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val # 값 복원
        it.iternext()   

    return grad

class XorLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = np.random.randn(input_size, hidden_size) / np.sqrt(input_size)
        self.params['b1'] = np.zeros(hidden_size)
        xavier_init = np.sqrt(output_size)
        self.params['W2'] = np.random.randn(hidden_size, output_size) / np.sqrt(hidden_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = a2
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return mean_squared_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads
    
network = XorLayerNet(input_size=2, hidden_size=10, output_size=1)
x_train =  np.array([[0,0],[0,1],[1,0],[1,1]])
t_train = np.array([0,1,1,0])

# 하이퍼파라미터
iters_num = 50000  # 반복 횟수를 적절히 설정한다.
train_size = 4
batch_size = 1   # 미니배치 크기
learning_rate = 0.01

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)
batch_mask = np.random.choice(train_size, batch_size)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)

    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    #grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    #print("loss : %0.005f"+ loss + " x_batch" + x_batch + " predict = "+ network.predict(x_batch) + " correct = "% t_batch)
    print("-------------")
    print(x_batch)
    print(loss)
    print(network.predict(x_batch))
    #print(t_batch)
    print("-------------")
    train_loss_list.append(loss)
    




-------------
[[0 0]]
0.403222750538
[[-0.89802311]]
-------------
-------------
[[0 1]]
2.10077747301
[[-1.04976949]]
-------------
-------------
[[1 0]]
1.81800530966
[[-0.90683261]]
-------------
-------------
[[1 0]]
1.6582674431
[[-0.8211356]]
-------------
-------------
[[1 0]]
1.51278614813
[[-0.73941723]]
-------------
-------------
[[1 1]]
0.365210723461
[[-0.85464697]]
-------------
-------------
[[0 0]]
0.143606861315
[[-0.53592324]]
-------------
-------------
[[0 0]]
0.133611056204
[[-0.51693531]]
-------------
-------------
[[0 0]]
0.124313175567
[[-0.49862446]]
-------------
-------------
[[1 0]]
1.21874773858
[[-0.56124805]]
-------------
-------------
[[0 1]]
1.2217544994
[[-0.56317273]]
-------------
-------------
[[0 1]]
1.11914193088
[[-0.49608952]]
-------------
-------------
[[1 1]]
0.143902016445
[[-0.5364737]]
-------------
-------------
[[0 1]]
0.989838542101
[[-0.40700998]]
-------------
-------------
[[0 0]]
0.0257106195578
[[-0.22676252]]
-------------
-----

[[ 0.55204834]]
-------------
-------------
[[0 1]]
0.102196618882
[[ 0.5479013]]
-------------
-------------
[[0 0]]
0.176829770403
[[ 0.59469281]]
-------------
-------------
[[0 0]]
0.164590070517
[[ 0.57374223]]
-------------
-------------
[[0 0]]
0.153198472343
[[ 0.55353134]]
-------------
-------------
[[1 1]]
0.113329204274
[[ 0.47608656]]
-------------
-------------
[[0 1]]
0.134896278333
[[ 0.48058441]]
-------------
-------------
[[1 1]]
0.113132593915
[[ 0.4756734]]
-------------
-------------
[[1 0]]
0.0963232017689
[[ 0.56108497]]
-------------
-------------
[[1 1]]
0.11136291095
[[ 0.47193837]]
-------------
-------------
[[1 1]]
0.100370590092
[[ 0.44804149]]
-------------
-------------
[[0 0]]
0.122007185422
[[ 0.49397811]]
-------------
-------------
[[0 1]]
0.157850687365
[[ 0.43812691]]
-------------
-------------
[[0 1]]
0.144654099278
[[ 0.46212622]]
-------------
-------------
[[0 0]]
0.134386859157
[[ 0.51843391]]
-------------
-------------
[[0 1]]
0.1427521338

0.179590828725
[[ 0.59931766]]
-------------
-------------
[[1 1]]
0.14851186147
[[ 0.54499883]]
-------------
-------------
[[1 1]]
0.133867331539
[[ 0.51743083]]
-------------
-------------
[[1 0]]
0.0846443547676
[[ 0.58855291]]
-------------
-------------
[[1 0]]
0.077293508042
[[ 0.60682445]]
-------------
-------------
[[1 1]]
0.139669008869
[[ 0.52852438]]
-------------
-------------
[[1 0]]
0.0802072535406
[[ 0.5994822]]
-------------
-------------
[[0 1]]
0.104200645116
[[ 0.5434901]]
-------------
-------------
[[0 0]]
0.166669463942
[[ 0.57735511]]
-------------
-------------
[[1 1]]
0.134246042795
[[ 0.51816222]]
-------------
-------------
[[0 1]]
0.116482013658
[[ 0.51733653]]
-------------
-------------
[[1 1]]
0.132074501207
[[ 0.51395428]]
-------------
-------------
[[0 0]]
0.142020919809
[[ 0.53295576]]
-------------
-------------
[[1 0]]
0.0942521669398
[[ 0.56582914]]
-------------
-------------
[[0 0]]
0.141085262306
[[ 0.53119726]]
-------------
-------------
[[0

0.118156391629
[[ 0.48612013]]
-------------
-------------
[[1 1]]
0.0930112425109
[[ 0.43130324]]
-------------
-------------
[[0 1]]
0.15812205683
[[ 0.43764414]]
-------------
-------------
[[0 0]]
0.111714858997
[[ 0.47268353]]
-------------
-------------
[[0 1]]
0.154604573211
[[ 0.44393422]]
-------------
-------------
[[0 1]]
0.14170172032
[[ 0.4676435]]
-------------
-------------
[[1 0]]
0.0988224032614
[[ 0.55542739]]
-------------
-------------
[[0 0]]
0.132514426448
[[ 0.51480953]]
-------------
-------------
[[0 0]]
0.123358271038
[[ 0.49670569]]
-------------
-------------
[[1 0]]
0.107498139842
[[ 0.53632309]]
-------------
-------------
[[0 1]]
0.129576837001
[[ 0.49092862]]
-------------
-------------
[[1 0]]
0.0888947153652
[[ 0.57834916]]
-------------
-------------
[[0 0]]
0.142156878243
[[ 0.5332108]]
-------------
-------------
[[0 0]]
0.132334148367
[[ 0.51445923]]
-------------
-------------
[[1 1]]
0.109814890678
[[ 0.46864676]]
-------------
-------------
[[1 

[[0 0]]
0.11160169678
[[ 0.47244406]]
-------------
-------------
[[1 1]]
0.0979926409507
[[ 0.44270225]]
-------------
-------------
[[0 1]]
0.159633208795
[[ 0.43496335]]
-------------
-------------
[[1 0]]
0.109194382596
[[ 0.53267916]]
-------------
-------------
[[0 0]]
0.113952672409
[[ 0.47739433]]
-------------
-------------
[[0 1]]
0.145287869191
[[ 0.46094922]]
-------------
-------------
[[0 0]]
0.115723250928
[[ 0.48108887]]
-------------
-------------
[[0 0]]
0.107732188473
[[ 0.46418141]]
-------------
-------------
[[1 1]]
0.0948908251128
[[ 0.43563936]]
-------------
-------------
[[0 0]]
0.0924460920382
[[ 0.42999091]]
-------------
-------------
[[1 1]]
0.0786355127612
[[ 0.39657411]]
-------------
-------------
[[0 0]]
0.0794441452881
[[ 0.39860794]]
-------------
-------------
[[1 0]]
0.150802594737
[[ 0.45081407]]
-------------
-------------
[[1 0]]
0.137738639438
[[ 0.47514071]]
-------------
-------------
[[0 1]]
0.165594214609
[[ 0.42451027]]
-------------
-----

...

0.0518851829885
[[ 0.67786592]]
-------------
-------------
[[0 0]]
0.051114348639
[[ 0.31973223]]
-------------
-------------
[[1 1]]
0.118265434324
[[ 0.48634439]]
-------------
-------------
[[1 1]]
0.106446386061
[[ 0.46140305]]
-------------
-------------
[[1 0]]
0.066545429855
[[ 0.6351838]]
-------------
-------------
[[0 1]]
0.0695432545776
[[ 0.62705696]]
-------------
-------------
[[0 0]]
0.0445799676344
[[ 0.29859661]]
-------------
-------------
[[1 1]]
0.106140975123
[[ 0.46074065]]
-------------
-------------
[[1 1]]
0.0955276210727
[[ 0.43709866]]
-------------
-------------
[[1 1]]
0.0859707121276
[[ 0.4146582]]
-------------
-------------
[[0 0]]
0.0275115866566
[[ 0.23457019]]
-------------
-------------
[[1 0]]
0.08609874962
[[ 0.58503313]]
-------------
-------------
[[1 0]]
0.0783544835328
[[ 0.60413517]]
-------------
-------------
[[0 1]]
0.081393466671
[[ 0.59653137]]
-------------
-------------
[[1 0]]
0.0648304102378
[[ 0.63991554]]
-------------
------------

-------------
-------------
[[0 1]]
0.0828727031368
[[ 0.59288158]]
-------------
-------------
[[1 1]]
0.092056440471
[[ 0.42908377]]
-------------
-------------
[[1 0]]
0.0773355330358
[[ 0.60671757]]
-------------
-------------
[[0 1]]
0.0768046179413
[[ 0.60806986]]
-------------
-------------
[[1 0]]
0.0641108575013
[[ 0.6419194]]
-------------
-------------
[[0 1]]
0.0642744590551
[[ 0.64146281]]
-------------
-------------
[[0 0]]
0.0439771003767
[[ 0.29657073]]
-------------
-------------
[[0 1]]
0.0625107946685
[[ 0.64641608]]
-------------
-------------
[[0 0]]
0.0448433772623
[[ 0.29947747]]
-------------
-------------
[[1 0]]
0.0556526718074
[[ 0.66637545]]
-------------
-------------
[[0 0]]
0.045516879647
[[ 0.30171801]]
-------------
-------------
[[1 0]]
0.0544258872795
[[ 0.67007308]]
-------------
-------------
[[0 1]]
0.0549553565804
[[ 0.66847215]]
-------------
-------------
[[1 1]]
0.12808642717
[[ 0.50613521]]
-------------
-------------
[[1 1]]
0.115271415005
[[

[[1 1]]
0.135441703272
[[ 0.52046461]]
-------------
-------------
[[0 1]]
0.0509045089249
[[ 0.68092475]]
-------------
-------------
[[0 0]]
0.0497361331218
[[ 0.31539224]]
-------------
-------------
[[0 1]]
0.0501228870666
[[ 0.68338387]]
-------------
-------------
[[0 0]]
0.0499782100344
[[ 0.31615885]]
-------------
-------------
[[0 0]]
0.0463943382309
[[ 0.30461234]]
-------------
-------------
[[0 0]]
0.0430675834116
[[ 0.29348793]]
-------------
-------------
[[0 1]]
0.0568108419431
[[ 0.66292184]]
-------------
-------------
[[0 0]]
0.0437634622003
[[ 0.2958495]]
-------------
-------------
[[1 1]]
0.115048511869
[[ 0.4796843]]
-------------
-------------
[[0 0]]
0.0352855311137
[[ 0.26565215]]
-------------
-------------
[[0 0]]
0.0327554628952
[[ 0.25595102]]
-------------
-------------
[[0 1]]
0.0706024085476
[[ 0.62422771]]
-------------
-------------
[[0 1]]
0.0643400906811
[[ 0.6412798]]
-------------
-------------
[[0 1]]
0.0586347347698
[[ 0.6575537]]
-------------


[[ 0.51691524]]
-------------
-------------
[[1 0]]
0.048436765404
[[ 0.68875487]]
-------------
-------------
[[1 0]]
0.0440644495958
[[ 0.70313488]]
-------------
-------------
[[0 0]]
0.0493223896296
[[ 0.31407766]]
-------------
-------------
[[0 0]]
0.0457802303598
[[ 0.30258959]]
-------------
-------------
[[0 0]]
0.0424925795701
[[ 0.29152214]]
-------------
-------------
[[1 1]]
0.116879454856
[[ 0.4834862]]
-------------
-------------
[[0 1]]
0.0598620866933
[[ 0.65398819]]
-------------
-------------
[[1 0]]
0.0532347616476
[[ 0.67370332]]
-------------
-------------
[[1 0]]
0.0484286119633
[[ 0.68878107]]
-------------
-------------
[[1 1]]
0.127534750438
[[ 0.50504406]]
-------------
-------------
[[0 0]]
0.0390635795638
[[ 0.27951236]]
-------------
-------------
[[1 1]]
0.109550103855
[[ 0.46808141]]
-------------
-------------
[[1 0]]
0.0627164569431
[[ 0.64583491]]
-------------
-------------
[[1 0]]
0.0570512246842
[[ 0.66220946]]
-------------
-------------
[[1 1]]
0

KeyboardInterrupt: 

In [110]:
print("test!!! start")
x_test = np.array([[1,1],[0,1]])
t_test = np.array([0,1])
print(network.predict([1,1]))
print("correct is 0")
print(network.predict(x_test[1]))
print("correct is 1")

test!!! start
[  1.14848897e-05]
correct is 0
[ 0.99998957]
correct is 1


In [112]:
print(network.predict([[0,1]]))

[[ 0.99998957]]


In [113]:
print(network.predict([[1,0]]))

[[ 0.99998753]]


In [114]:
print(network.predict([[0,0]]))

[[  5.73215724e-06]]


In [115]:
print(network.predict([[1,1]]))

[[  1.14848897e-05]]
